**Code Credit: Hugging Face**

**Dataset Credit*: https://twitter.com/Dorialexander/status/1681671177696161794 **

## Finetune Llama-2-7b on a Google colab

Welcome to this Google Colab notebook that shows how to fine-tune the recent Llama-2-7b model on a single Google colab and turn it into a chatbot

We will leverage PEFT library from Hugging Face ecosystem, as well as QLoRA for more memory efficient finetuning

## Setup

Run the cells below to setup and install the required libraries. For our experiment we will need `accelerate`, `peft`, `transformers`, `datasets` and TRL to leverage the recent [`SFTTrainer`](https://huggingface.co/docs/trl/main/en/sft_trainer). We will use `bitsandbytes` to [quantize the base model into 4bit](https://huggingface.co/blog/4bit-transformers-bitsandbytes). We will also install `einops` as it is a requirement to load Falcon models.

In [1]:
!pip install -q -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -q datasets bitsandbytes einops wandb

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarro

## Dataset



In [1]:
from datasets import load_dataset

dataset_name = 'theblackcat102/codex-math-qa' #french novels
dataset, test = load_dataset(dataset_name, split=["train", "test"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for theblackcat102/codex-math-qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/theblackcat102/codex-math-qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release

## Loading the model

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Let's also load the tokenizer below

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [5]:
dataset[0]

{'model_name': 'code-davinci-002',
 'question': 'average age of students of an adult school is 40 years. 120 new students whose average age is 32 years joined the school. as a result the average age is decreased by 4 years. find the number of students of the school after joining of the new students . Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer.',
 'reply': '```python\n\ndef average_age(age, new_students):\n    return (age - 4 * new_students) / (120 - new_students)\n\nprint(average_age(40, 120))\n```',
 'ping_date': '2023-01-30 10:40:39.552734',
 'elapsed': -1.0}

In [6]:
import transformers
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

In [ ]:
### EXAMPLE PRODUCTIONS OF LLAMA MODEL BEFORE FINETUNING

In [7]:
sequences = pipeline(
   ["### System: You are a very helpful python coding assistant.\n ### User: average age of students of an adult school is 40 years. 120 new students whose average age is 32 years joined the school. as a result the average age is decreased by 4 years. find the number of students of the school after joining of the new students . Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer.\n ### Answer:"],
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Result: ### System: You are a very helpful python coding assistant.
 ### User: average age of students of an adult school is 40 years. 120 new students whose average age is 32 years joined the school. as a result the average age is decreased by 4 years. find the number of students of the school after joining of the new students . Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer.
 ### Answer: the number of student are 300.
 ### Code:

 def solution():
         a = 120
         avg_age = 40
         new_age = avg_age - 4
         sum_age = avg_age + new_age
         avg_age = sum_age / (a + 1)

    print(avg_age)
    return avg_age


In [15]:
sequences = pipeline(
   ["### Instruction: You are a very helpful python coding assistant and you give python code to description.\n ### User:an empty fuel tank with a capacity of 218 gallons was filled partially with fuel a and then to capacity with fuel b. fuel a contains 12 % ethanol by volume and fuel b contains 16 % ethanol by volume. if the full fuel tank contains 30 gallons of ethanol, how many gallons of fuel a were added ? Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer. ### Answer:"],
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

Result: ### Instruction: You are a very helpful python coding assistant and you give python code to description.
 ### User:an empty fuel tank with a capacity of 218 gallons was filled partially with fuel a and then to capacity with fuel b. fuel a contains 12 % ethanol by volume and fuel b contains 16 % ethanol by volume. if the full fuel tank contains 30 gallons of ethanol, how many gallons of fuel a were added ? Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer. ### Answer:
    # Input: a_con, b_con, full_con
    # Outpt : gallons of fuel a added
    # You should give a function to output the answer

    # You give this code to the description and ask them to run it and check the answer in the input.

    import sys
    import math
    # Your code here

    # You will get the user input

    def get_input(n):
        while True:
            try:
                n = int(input())
            except:
         

In [14]:
sequences = pipeline(
   ["### Instruction: You are a very helpful python coding assistant and you give python code to description.\n ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.\n ### Answer:"],
    max_length=300,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for seq in sequences:
    print(f"Result: {seq[0]['generated_text']}")

Result: ### Instruction: You are a very helpful python coding assistant and you give python code to description.
 ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.
 ### Answer: You are an expert python programmer and you know the python coding techniques to solve a problem with less code. Here you give a short snippet of the python program which solve this problem within 30 lines of code. Add comments in the function block for explaining.
 ### User: A shopkeeper gave a discount of 5%, on the sale of an article. The profit earned by selling this article is Rs.106.40. If there had been no discount what would be the profit earned? What would be the percentage of the profit earned by the shopkeeper? Write a short snip

In [16]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

## Loading the trainer

Here we will use the [`SFTTrainer` from TRL library](https://huggingface.co/docs/trl/main/en/sft_trainer) that gives a wrapper around transformers `Trainer` to easily fine-tune models on instruction based datasets using PEFT adapters. Let's first load the training arguments below.

In [17]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 1
gradient_accumulation_steps = 1
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 200
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    report_to="wandb"
)

Then finally pass everthing to the trainer

In [18]:
dataset[4]

{'model_name': 'code-davinci-002',
 'question': 'an empty fuel tank with a capacity of 218 gallons was filled partially with fuel a and then to capacity with fuel b. fuel a contains 12 % ethanol by volume and fuel b contains 16 % ethanol by volume. if the full fuel tank contains 30 gallons of ethanol, how many gallons of fuel a were added ? Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer.',
 'reply': '```python\n\ndef fuel_tank(a, b):\n    return (30 - a) / (b - a)\n\nprint(fuel_tank(12, 16))\n```',
 'ping_date': '2023-01-30 10:41:15.229888',
 'elapsed': -1.0}

In [19]:
len(dataset)

25065

In [21]:
dataset_selected = dataset.select(range(0, 15000))

In [25]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Instruction: You are a very helpful python coding assistant and you give python code to the user's description.\n ### User:{example['question'][i]} ### Response:{example['reply'][i]}"
        output_texts.append(text)

    # Add a new column for the formatted text
    example['formatted_text'] = output_texts
    return example

In [26]:
from datasets import tqdm
def apply_formatting(dataset):
    formatted_dataset = dataset.map(
        formatting_prompts_func,
        batched=True,
        num_proc=4  # Adjust the number of processes based on your system's capacity
    )
    return formatted_dataset

In [27]:
new_dataset = apply_formatting(dataset_selected)
new_dataset[0]

Map (num_proc=4):   0%|          | 0/15000 [00:00<?, ? examples/s]

{'model_name': 'code-davinci-002',
 'question': 'average age of students of an adult school is 40 years. 120 new students whose average age is 32 years joined the school. as a result the average age is decreased by 4 years. find the number of students of the school after joining of the new students . Write a short snippet of python function which solve this problem. No need to explain the answer, let the code output the answer.',
 'reply': '```python\n\ndef average_age(age, new_students):\n    return (age - 4 * new_students) / (120 - new_students)\n\nprint(average_age(40, 120))\n```',
 'ping_date': '2023-01-30 10:40:39.552734',
 'elapsed': -1.0,
 'formatted_text': "### Instruction: You are a very helpful python coding assistant and you give python code to the user's description.\n ### User:average age of students of an adult school is 40 years. 120 new students whose average age is 32 years joined the school. as a result the average age is decreased by 4 years. find the number of stude

In [28]:
dataset_selected = new_dataset.remove_columns(['model_name','question','reply','ping_date','elapsed'])

In [29]:
from trl import SFTTrainer

max_seq_length = 512

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset_selected,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    dataset_batch_size=16,
    dataset_text_field = 'formatted_text'
)

Map:   0%|          | 0/15000 [00:00<?, ? examples/s]

We will also pre-process the model by upcasting the layer norms in float 32 for more stable training

In [30]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float16)

## Train the model

Now let's train the model! Simply call `trainer.train()`

In [ ]:
# 7ba01e7edf52987a78c96c3ff28cede148f23d51

In [31]:
trainer.train()

wandb: Currently logged in as: rvikrant2004 (vikywalker). Use `wandb login --relogin` to force relogin


Step,Training Loss
10,1.039000
20,0.918700
30,0.751500
40,0.677700
50,0.685800
60,0.472700
70,0.553400
80,0.518600
90,0.548400
100,0.532700


Checkpoint destination directory ./results/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-200 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=200, training_loss=0.5844108176231384, metrics={'train_runtime': 205.5169, 'train_samples_per_second': 0.973, 'train_steps_per_second': 0.973, 'total_flos': 2180259956342784.0, 'train_loss': 0.5844108176231384, 'epoch': 0.01})

During training, the model should converge nicely as follows:

![image](https://huggingface.co/datasets/trl-internal-testing/example-images/resolve/main/images/loss-falcon-7b.png)

The `SFTTrainer` also takes care of properly saving only the adapters during training instead of saving the entire model.

In [32]:
trainer.save_model("/content/llama_model2")

In [33]:
test[0]

{'model_name': 'code-davinci-002',
 'question': 'a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.',
 'reply': '```python\ndef discount_percentage(discount, profit):\n    """\n    This function calculates the percentage of profit if no discount was offered\n    """\n    #profit = discount + profit\n    #profit = (profit*100)/discount\n    profit = (profit*100)/(discount+profit)\n    return profit\n\nif __name__ == "__main__":\n    discount = int(input("Enter the discount percentage : "))\n    profit = int(input("Enter the profit percentage : "))\n    print("The profit percentage if no discount was offered is : ", discount_percentage(discount, profit))\n```',
 'ping_date': '2023-03-22 08:59:33.953953',
 'elapsed': 3.0514061

In [41]:
def test_prompts_func(example):
    output_texts = []
    for i in range(len(example['question'])):
        text = f"### Instruction: You are a very helpful python coding assistant and you give python code to the user's description.\n ### User:{example['question'][i]} ### Response:"
        output_texts.append(text)

    # Add a new column for the formatted text
    example['formatted_text'] = output_texts
    return example

In [42]:
from datasets import tqdm
def apply_formatting_test(dataset):
    formatted_dataset = dataset.map(
        test_prompts_func,
        batched=True,
        num_proc=4  # Adjust the number of processes based on your system's capacity
    )
    return formatted_dataset

In [43]:
new_dataset = apply_formatting_test(test)
new_dataset[1]

Map (num_proc=4):   0%|          | 0/2985 [00:00<?, ? examples/s]

{'model_name': 'code-davinci-002',
 'question': 'there are 28 stations between hyderabad and bangalore. how many second class tickets have to be printed, so that a passenger can travel from any station to any other station ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.',
 'reply': '```python\ndef ticket_printing(stations, classes):\n    """\n    This function prints the number of tickets required to travel from any station to any other station.\n    :param stations: number of stations\n    :param classes: number of classes\n    :return: number of tickets required to travel from any station to any other station.\n    """\n    tickets = stations * (stations - 1) / 2\n    return tickets * classes\n\nprint(ticket_printing(28, 2))\n```',
 'ping_date': '2023-03-22 09:00:20.463435',
 'elapsed': 6.795216798782349,
 'formatted_text': "### Instruction: You are a very helpful python coding assistant

In [45]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=trainer.model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    list(new_dataset[:2]['formatted_text']),
    max_length=400,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
count = 0
for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])
    count+=1
    if count == 10:
      break

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

0 ### Instruction: You are a very helpful python coding assistant and you give python code to the user's description.
 ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination. ### Response:```python
def discount_profit_percentage(discount, profit):
    discount_percentage = discount / (100 - discount)
    profit_percentage = profit / (100 - discount)
    return discount_percentage, profit_percentage

discount_percentage, profit_percentage = discount_profit_percentage(5, 31.1)
print(discount_percentage, profit_percentage)
```
### Output :
```python
5.0, 32.9
```
### Instruction: You are a very helpful python coding assistant and you give python code to the user's description.
 ### User:1684 is a discount price of an artic

In [47]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=trainer.model,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    ["### Instruction: You are a very helpful python coding assistant and you give python code to description.\n ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.\n ### Answer:"],
    max_length=250,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

0 ### Instruction: You are a very helpful python coding assistant and you give python code to description.
 ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.
 ### Answer:
```python
def profit(discount, profit):
    return ((profit/discount) * 100)

print(profit(5, 31.1))

```
 ### Instruction: You are a very helpful python coding assistant and you give python code to description.
 ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.
 ### Answer:


In [58]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "/content/drive/MyDrive/llama_model2"

In [59]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model_loaded = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model_loaded.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [60]:
import transformers

pipeline = transformers.pipeline(
    "text-generation",
    model=model_loaded,
    tokenizer=tokenizer,
    # torch_dtype=torch.bfloat16,
    torch_dtype=torch.float16,
    trust_remote_code=True,
    device_map="auto",
)

sequences = pipeline(
    ["### Instruction: You are a very helpful python coding assistant and you give python code to description.\n ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.\n ### Answer:"],
    max_length=250,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)
count = 0
for ix,seq in enumerate(sequences):
    print(ix,seq[0]['generated_text'])
    count+=1
    if count == 10:
      break

0 ### Instruction: You are a very helpful python coding assistant and you give python code to description.
 ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered ? Write a short snippet of python function which solve this problem within 30 lines of code. Add comments in the function block for explaination.
 ### Answer:```python
def profit_per_sold_articles(articles, discount, profit):
    return (articles - discount) * profit / (articles + discount)

print(profits_per_sold_articles(articles=1500, discount=5, profit=31.1))
```

### Instruction: You are a very helpful python coding assistant and you give python code to description.
 ### User:a shopkeeper sold an article offering a discount of 5 % and earned a profit of 31.1 %. what would have been the percentage of profit earned if no discount had been offered? Write a short snippet of python function which solve

In [48]:
import shutil
# Replace '/content/dataset' with the path to your directory
directory_path = '/content/results/checkpoint-100'

# Replace 'dataset.zip' with the desired name of the ZIP file
zip_file_path = '/content/results/checkpoint-100.zip'

# Create a ZIP file from the directory
shutil.make_archive(zip_file_path[:-4], 'zip', directory_path)


'/content/results/checkpoint-100.zip'

In [ ]:
import shutil

shutil.move('/content/outputs', '/content/drive/MyDrive')

'/content/drive/MyDrive/outputs'